In [1]:
# ------------------------------------------------------------------------------
# Calculate the MOTIF Overlaps with SNPs using MOTIFBreakR

# Libraries

In [2]:
source('MS0_Libraries.r')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on '../conda_environment/Environment_Configs.csv'”


[1] "/home/icb/corinna.losert/miniconda3/envs/scgrn_R_4_1//lib/R/library"



Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t


Loading required package: limma

Loading required package: mgcv

Loading required package: nlme

This is mgcv 1.9-1. For overview type 'help("mgcv-package")'.

Loading required package: genefilter

Loading required package: BiocParallel

corrplot 0.95 loaded

Loading required package: viridisLite


Attaching package: ‘purrr’


The following object is masked from ‘package:data.table’:

    transpose



Attaching package: ‘MASS’


The following object is masked from ‘package:genefilter’:

    area


The following object is masked from ‘package:biomaRt’:

    select



Attaching package: ‘dplyr’


The following object is masked from ‘package:MASS’:

    select


The following object is masked from ‘package:nlme’:

    collapse


The following objects are masked from ‘package:data.table’:

    between, first, last


The following object is masked from ‘package:biomaRt’:

    selec

In [3]:
library(rtracklayer)
library(data.table)

In [4]:
library(BSgenome.Hsapiens.UCSC.hg38)


In [5]:
library(TFBSTools)


Attaching package: 'TFBSTools'


The following object is masked from 'package:Gviz':

    tags




In [6]:
library(JASPAR2020)


In [7]:
library(Biostrings)

In [8]:
library(GenomicRanges)

In [9]:
library(stringr)

In [10]:
source("snipe.R")


Attaching package: 'httr'


The following object is masked from 'package:Biobase':

    content



Attaching package: 'magrittr'


The following object is masked from 'package:AnnotationFilter':

    not


The following object is masked from 'package:tidyr':

    extract


The following object is masked from 'package:purrr':

    set_names




In [11]:
source('MS4_Plot_Config.r')

Warning message:
"The `size` argument of `element_line()` is deprecated as of ggplot2 3.4.0.
ℹ Please use the `linewidth` argument instead."


In [12]:
source('MS1_Functions.r')

In [14]:
packageVersion('motifbreakR')

[1] '2.8.0'

# Parameters

In [30]:
### Define which co-eQTL mapping results to use

In [31]:
coeqtl_results_path = 'co_qtls_decision_tree/'   # path to results of full mapping
coeqtl_results_path_old = 'co_qtls_sceqtlgen/'   # path to initial mapping (Evaluation)

In [32]:
### Path to the co-EQTL Files

In [33]:
data_path = '../data/current/coeqtl_mapping/'

In [ ]:
### Path to the eQTL Files (sceQTLGen)

In [34]:
data_path_eqtl = '../data/current/eqtl_mapping/'

In [35]:
eqtl_results = 'sc_eqtlgen_final'

In [ ]:
### Path to meta data files

In [40]:
data_path_meta = '../data/current/meta-data'

In [41]:
### How to identify sign co-eqtls (parameters)

In [43]:
p_val_column = 'p_val_mt_eGene' # p_value # nominal , #p_val_mt_eGene
mt_method = 'eGene_bf'

In [44]:
p_value_sign = 0.05

In [45]:
n_sample_filter = 133

# Data

## Load significant coeqtls

In [ ]:
### Annotated file from 00 Data Annotation script

In [49]:
meta_results = fread(paste0( "../data/current/coeqtl_mapping/", coeqtl_results_path , "/shared_results/", '5DS_Meta_Analysis_Sign_coeQTLs', n_sample_filter, p_val_column  ,'.csv'))

In [50]:
head(meta_results,2)

V1,eGene,coeGene,snp_id,triplet,cell_type,dataset,feature_id,n_samples,p_val_mt,⋯,z_score;oneK1K;CD4_T,z_score;oneK1K;CD8_T,z_score;oneK1K;Mono,z_score;oneK1K;NK,z_score;wijst;B,z_score;wijst;CD4_T,z_score;wijst;CD8_T,z_score;wijst;Mono,z_score;wijst;NK,remap_overlap_tf_snp
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,A2M,BACH2,12:8971336:T:C,12:8971336:T:C;A2M_BACH2,CD4_T,meta_5ds,A2M_BACH2,977,0.004284312,⋯,-6.281251,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,A2M,BACH2,12:8992460:T:C,12:8992460:T:C;A2M_BACH2,CD4_T,meta_5ds,A2M_BACH2,977,0.026783812,⋯,-5.991429,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [ ]:
### Combine non-ribosomal/ non-mitochondrial columns

In [52]:
meta_results$n_rb_mt = 0
meta_results$n_rb_mt[(meta_results$eGene_rib ==0) & (meta_results$coeGene_rib == 0) & (meta_results$eGene_mt == 0) & (meta_results$coeGene_mt == 0)] = 1

In [53]:
meta_results$snp_pos = str_replace(meta_results$snp_id ,".*:(?=\\d)", "")
meta_results$snp_pos = str_replace(meta_results$snp_pos , ":.*", "")
meta_results$snp_chr = str_replace(meta_results$snp_id , ':.*', '')

In [54]:
head(meta_results,2)

V1,eGene,coeGene,snp_id,triplet,cell_type,dataset,feature_id,n_samples,p_val_mt,⋯,z_score;oneK1K;NK,z_score;wijst;B,z_score;wijst;CD4_T,z_score;wijst;CD8_T,z_score;wijst;Mono,z_score;wijst;NK,remap_overlap_tf_snp,n_rb_mt,snp_pos,snp_chr
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>
1,A2M,BACH2,12:8971336:T:C,12:8971336:T:C;A2M_BACH2,CD4_T,meta_5ds,A2M_BACH2,977,0.004284312,⋯,NA,NA,NA,NA,NA,NA,NA,1,8971336,12
2,A2M,BACH2,12:8992460:T:C,12:8992460:T:C;A2M_BACH2,CD4_T,meta_5ds,A2M_BACH2,977,0.026783812,⋯,NA,NA,NA,NA,NA,NA,NA,1,8992460,12


## Load Finemapped SNPs 

In [1]:
### Get SNPs from co-eQTL Finemapping

In [ ]:
## Define dataset (meta-analysis) and cell-types

In [59]:
dataset_alt = 'meta_analysis/meta_5ds'

In [60]:
cell_types = c('Mono', 'CD8_T', 'NK', 'B', 'CD4_T')

In [ ]:
### Define which finemapping version to use

In [ ]:
finemapping_version = 'lenient' # lenient or strict

In [ ]:
### Empty dataframe to save all finemapping results

In [61]:
finemapping_results = data.frame()

In [ ]:
### Load finemapping results

In [76]:
if(finemapping_version == 'lenient'){
    files = list.files(paste0(data_path,coeqtl_results_path ,dataset_alt, '/Finemaping_lenient/NT/'))
    files = files[!is.na(str_extract(files, 'newThreshold'))]

    coeqtl_all_files = files


    coeqtl_all_effect = data.frame()

    for(i in coeqtl_all_files){
        print(i)
        flush.console()
        file = paste0(data_path, coeqtl_results_path ,dataset_alt, '/', '/Finemaping_lenient/NT/', i)
        print(file.info(file)$ctime)
        data1 = fread(file)
        data1 = as.data.frame(data1)
        cell_type_var = str_extract(i, 'B|NK|Mono|CD4_T|CD8_T')
        data1$cell_type = cell_type_var
        data1$file_name = i
        data1$SusieRss_CS_G = NULL
        #data1$`n_samples;Multiome` = NULL
        #data1$`n_e_samples;Multiome` = NULL
        #data1$`z_score;Multiome` = NULL
        coeqtl_all_effect = rbind(data1, coeqtl_all_effect)
    }

    finemapping_results = rbind(finemapping_results, coeqtl_all_effect)
    }


[1] "meta_joint_susie_finemap_B_1_newThreshold_formatted.txt.gz"
[1] "2025-06-04 21:13:25 CEST"
[1] "meta_joint_susie_finemap_B_11_newThreshold_formatted.txt.gz"
[1] "2025-06-04 21:13:25 CEST"
[1] "meta_joint_susie_finemap_B_12_newThreshold_formatted.txt.gz"
[1] "2025-06-04 21:13:28 CEST"
[1] "meta_joint_susie_finemap_B_14_newThreshold_formatted.txt.gz"
[1] "2025-06-04 21:13:27 CEST"
[1] "meta_joint_susie_finemap_B_15_newThreshold_formatted.txt.gz"
[1] "2025-06-04 21:13:28 CEST"
[1] "meta_joint_susie_finemap_B_17_newThreshold_formatted.txt.gz"
[1] "2025-06-04 21:13:27 CEST"
[1] "meta_joint_susie_finemap_B_18_newThreshold_formatted.txt.gz"
[1] "2025-06-04 21:13:26 CEST"
[1] "meta_joint_susie_finemap_B_19_newThreshold_formatted.txt.gz"
[1] "2025-06-04 21:13:26 CEST"
[1] "meta_joint_susie_finemap_B_2_newThreshold_formatted.txt.gz"
[1] "2025-06-04 21:13:25 CEST"
[1] "meta_joint_susie_finemap_B_20_newThreshold_formatted.txt.gz"
[1] "2025-06-04 21:13:26 CEST"
[1] "meta_joint_susie_finemap_B_

In [44]:
#### Strict grouping version
if(finemapping_version == 'strict'){
    for(j in cell_types){
        cell_type_var = j
        coeqtl_all_files = list.files(paste0(data_path, coeqtl_results_path ,dataset_alt, '/',  cell_type_var , '/Full_Window_Results/'))
        coeqtl_all_files = coeqtl_all_files[!is.na(str_extract(coeqtl_all_files, cell_type_var))]
        coeqtl_all_files = coeqtl_all_files[!is.na(str_extract(coeqtl_all_files, '.txt'))]


        coeqtl_all_effect = data.frame()

        for(i in coeqtl_all_files){
            print(i)
            flush.console()
            file = paste0(data_path, coeqtl_results_path ,dataset_alt, '/',  cell_type_var , '/Full_Window_Results/', i)
            print(file.info(file)$ctime)
            data1 = fread(file)
            data1 = as.data.frame(data1)
            data1$cell_type = cell_type_var
            data1$file_name = i
            data1$SusieRss_CS_G = NULL
            #data1$`n_samples;Multiome` = NULL
            #data1$`n_e_samples;Multiome` = NULL
            #data1$`z_score;Multiome` = NULL
            coeqtl_all_effect = rbind(data1, coeqtl_all_effect)
        }

        finemapping_results = rbind(finemapping_results, coeqtl_all_effect)
        }
    }

[1] "meta_joint_susie_finemap_Mono_1_formatted.txt"
[1] "2025-02-26 15:59:09 CET"
[1] "meta_joint_susie_finemap_Mono_10_formatted.txt"
[1] "2025-02-26 15:59:06 CET"
[1] "meta_joint_susie_finemap_Mono_11_formatted.txt"
[1] "2025-02-26 15:59:07 CET"
[1] "meta_joint_susie_finemap_Mono_12_formatted.txt"
[1] "2025-02-26 15:59:07 CET"
[1] "meta_joint_susie_finemap_Mono_13_formatted.txt"
[1] "2025-02-26 15:59:08 CET"
[1] "meta_joint_susie_finemap_Mono_15_formatted.txt"
[1] "2025-02-26 15:59:08 CET"
[1] "meta_joint_susie_finemap_Mono_16_formatted.txt"
[1] "2025-02-26 15:59:08 CET"
[1] "meta_joint_susie_finemap_Mono_17_formatted.txt"
[1] "2025-02-26 15:59:08 CET"
[1] "meta_joint_susie_finemap_Mono_18_formatted.txt"
[1] "2025-02-26 15:59:08 CET"
[1] "meta_joint_susie_finemap_Mono_19_formatted.txt"
[1] "2025-02-26 15:59:09 CET"
[1] "meta_joint_susie_finemap_Mono_2_formatted.txt"
[1] "2025-02-26 15:59:09 CET"
[1] "meta_joint_susie_finemap_Mono_20_formatted.txt"
[1] "2025-02-26 15:59:09 CET"
[1] "m

In [77]:
head(finemapping_results,2)

,coQTL,eGene,feature_chromosome,feature_start,feature_end,coeGene,feature_id,snp_id,snp_chromosome,snp_position,⋯,SusieRss_pip,SusieRss_CS,SusieRss_ResVar,lbf_cs_1,lbf_cs_2,lbf_cs_3,lbf_cs_4,lbf_cs_5,cell_type,file_name
,<chr>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<dbl>,⋯,<dbl>,<int>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,9:135985420:T:C;PTGDS_SELL,PTGDS,9,136975092,136981742,SELL,PTGDS_SELL,9:135985420:T:C,9,135985420,⋯,0.007615623,NA,TRUE,-1.004602,-0.3033094,-0.04461793,-0.04147275,-0.03254301,NK,meta_joint_susie_finemap_NK_9_newThreshold_formatted.txt.gz
2,9:135986302:G:C;PTGDS_SELL,PTGDS,9,136975092,136981742,SELL,PTGDS_SELL,9:135986302:G:C,9,135986302,⋯,0.007823508,NA,TRUE,-0.957815,-0.2567398,-0.02082910,-0.01837484,-0.01163311,NK,meta_joint_susie_finemap_NK_9_newThreshold_formatted.txt.gz


In [81]:
unique(finemapping_results$cell_type)

[1] "NK"    "Mono"  "CD8_T" "CD4_T" "B"

In [83]:
unique(finemapping_results$SusieRss_CS)

[1] NA  1  2  3  5  4

In [ ]:
### Get the SNPs from the finemapping within a Credible Set

In [84]:
finemapping_snps = finemapping_results[!is.na(finemapping_results$SusieRss_CS),]

In [85]:
unique(finemapping_snps$SusieRss_CS)

[1] 1 2 3 5 4

In [86]:
length(unique(finemapping_snps$snp_id))

[1] 9717

In [87]:
finemapping_snps$snp_pos = str_replace(finemapping_snps$snp_id ,".*:(?=\\d)", "")
finemapping_snps$snp_pos = str_replace(finemapping_snps$snp_pos , ":.*", "")
finemapping_snps$snp_chr = str_replace(finemapping_snps$snp_id , ':.*', '')

In [88]:
head(finemapping_snps,2)

,coQTL,eGene,feature_chromosome,feature_start,feature_end,coeGene,feature_id,snp_id,snp_chromosome,snp_position,⋯,SusieRss_ResVar,lbf_cs_1,lbf_cs_2,lbf_cs_3,lbf_cs_4,lbf_cs_5,cell_type,file_name,snp_pos,snp_chr
,<chr>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<dbl>,⋯,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
282,9:136936281:G:C;PTGDS_SELL,PTGDS,9,136975092,136981742,SELL,PTGDS_SELL,9:136936281:G:C,9,136936281,⋯,TRUE,14.36794,-0.6699152,-0.2333248,-0.2248034,-0.1989274,NK,meta_joint_susie_finemap_NK_9_newThreshold_formatted.txt.gz,136936281,9
283,9:136936488:T:C;PTGDS_SELL,PTGDS,9,136975092,136981742,SELL,PTGDS_SELL,9:136936488:T:C,9,136936488,⋯,TRUE,14.07260,-0.6736336,-0.2362273,-0.2276421,-0.2015498,NK,meta_joint_susie_finemap_NK_9_newThreshold_formatted.txt.gz,136936488,9


In [89]:
finemapping_snps = unique(finemapping_snps[,c('snp_id', 'snp_pos', 'snp_chr')])

## Get SNP - eGene combinations used for testing (sceQTLGen)

In [ ]:
## Get eGene- finemapped SNPs from sceQTLGene used as input for co-eQTL mapping

In [90]:
cell_types = c('CD4_T', 'Mono', 'CD8_T', 'NK', 'B')

In [91]:
egenes = data.frame()

In [92]:
for(i in cell_types){
    print(i)
    snp_gene_mapping = read.csv(paste0(data_path_eqtl, eqtl_results , '/' , i, '_snp_egene_mapping.csv'))
    snp_gene_mapping$cell_type = i
    egenes = rbind(egenes, snp_gene_mapping)
    }
    

[1] "CD4_T"
[1] "Mono"
[1] "CD8_T"
[1] "NK"
[1] "B"


In [93]:
## Add the SNP and chr positions

In [94]:
egenes$snp_pos = str_replace(egenes$snp_id ,".*:(?=\\d)", "")
egenes$snp_pos = str_replace(egenes$snp_pos , ":.*", "")
egenes$snp_chr = str_replace(egenes$snp_id , ':.*', '')

In [95]:
head(egenes,2)

,X,feature_id,snp_id,feature_start,feature_end,p_value,beta,cell_type,snp_pos,snp_chr
,<int>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>
1,0,GSTM3,1:109740350:G:T,109733932,109741038,5.311981e-277,-1.1703583,CD4_T,109740350,1
2,1,GSTM3,1:109731286:G:T,109733932,109741038,1.872347e-74,-0.7292481,CD4_T,109731286,1


In [96]:
unique(egenes$cell_type)

[1] "CD4_T" "Mono"  "CD8_T" "NK"    "B"

In [97]:
length(unique(egenes$snp_id))

[1] 13744

In [98]:
#coegene_data

# Calculate Motif Overlaps (using MotifbreakR)

## Preparation

In [99]:
meta_results = as.data.frame(meta_results)

In [103]:
### Define which snps to run for
tested_snps = 'finemapping'   # alternatives: eSNPs, coeqtlSNPs

In [104]:
if(tested_snps == 'coeqtlSNPs'){
    snp_tests = unique(meta_results[,c ('snp_id', 'snp_pos', 'snp_chr'
                                       )])
    }

if(tested_snps == 'finemapping'){
    snp_tests = unique(finemapping_snps)
    }

if(tested_snps == 'eSNPs'){
    snp_tests = unique(egenes[,c('snp_id', 'snp_pos', 'snp_chr')])}

In [105]:
## Define p-value threshold to use for cutting of MotifBreakR results
sel_thres = 1e-4

In [106]:
sel_thres

[1] 1e-04

In [107]:
nrow(snp_tests)

[1] 9717

In [108]:
#co_egene_tfs$allele = str_replace(co_egene_tfs$snp_id, '.*:', '')
snp_tests$allele = str_replace(snp_tests$snp_id, '.*:', '')

In [109]:
head(snp_tests,2)

,snp_id,snp_pos,snp_chr,allele
,<chr>,<chr>,<chr>,<chr>
282,9:136936281:G:C,136936281,9,C
283,9:136936488:T:C,136936488,9,C


## Calculate the overlap with MOTIFS (motifbreakr)

In [16]:
#snp_tests = snp_tests[snp_tests$snp_id %in%  '7:28117268:C:T',]

In [18]:
## Information on  motifbreakr
# result intepretetation:
# pctRef, pctAlt: display the the score for the motif in the sequence as a percentage of the best score that motif could achieve on an ideal sequence


## Scoreing methods:
#The first is the standard summation of log probabilities (method='log'). 
# The second and third are the weighted sum and information content methods (method='default' and method='ic')

# For all three methods, motifBreakR scores and reports the reference and alternate alleles of the sequence 
# (F(sREF,M)nand F(sALT,M) ), and provides the matrix scores psREF and psALT of the SNP (or variant). 
# The scores are scaled as a fraction of scoring range 0-1 of the motif matrix

# invoking filterp=TRUE and setting a threshold at a desired p-value e.g 1e-4, we perform a rough filter on the results by rounding all values in the PWM to two decimal place
# with function calculate PValue actual result can b calculated

In [19]:
#head(data,2)

In [115]:
tested_snps

[1] "finemapping"

In [ ]:
### Get already executed SNPs to not redo again from saved location and/or initialize with empty dataframe

In [116]:
result_data = read.csv(paste0(data_path_meta, '/MOTIF_matches_meta_5ds_results_snps_breakr_280125',tested_snps, '_', sel_thres,  '.csv'))

In [114]:
# result_data = data.frame()

In [117]:
length(unique(result_data$SNP_id))

[1] 12034

In [118]:
result_data$X = NULL

In [119]:
result_data$X.1 = NULL

In [120]:
### Check SNPs for which already result
nrow(snp_tests[snp_tests$snp_id %in% str_replace(unique(result_data$SNP_id), 'chr', ''),])

### Only run for the ones not included yet
snp_tests = snp_tests[!snp_tests$snp_id %in% str_replace(unique(result_data$SNP_id), 'chr', ''),]

[1] 9633

In [122]:
head(snp_tests,2)

,snp_id,snp_pos,snp_chr,allele
,<chr>,<chr>,<chr>,<chr>
2864,9:126940688:A:G,126940688,9,G
124592,4:118254749:G:A,118254749,4,A


In [123]:
# SNPs for which to execute code

In [124]:
nrow(snp_tests)

[1] 84

In [105]:
## Check sources available for MotifBreakR

In [106]:
sources = data.frame(table(mcols(MotifDb)$organism, mcols(MotifDb)$dataSource))

In [107]:
sources[sources$Var1 == 'Hsapiens',]

,Var1,Var2,Freq
,<fct>,<fct>,<int>
19,Hsapiens,cisbp_1.02,313
79,Hsapiens,FlyFactorSurvey,0
139,Hsapiens,HOCOMOCOv10,640
199,Hsapiens,HOCOMOCOv11-core-A,181
259,Hsapiens,HOCOMOCOv11-core-B,84
319,Hsapiens,HOCOMOCOv11-core-C,135
379,Hsapiens,HOCOMOCOv11-secondary-A,46
439,Hsapiens,HOCOMOCOv11-secondary-B,19
499,Hsapiens,HOCOMOCOv11-secondary-C,13


In [ ]:
## Run MotifBreakR

In [108]:
for(i in 1:nrow(snp_tests)){
    data = snp_tests[i,]
    
    chr = paste0('chr', data$snp_chr)
    snp_position = as.numeric(data$snp_pos)
    snp = data$snp_id
    
    # Split the SNP ID to get individual components
    snp_info <- strsplit(snp, ":")[[1]]
    chromosome <- snp_info[1]
    position <- as.numeric(snp_info[2])
    ref <- snp_info[3]
    alt <- snp_info[4]
    
    # Create a GRanges object
    gr <- GRanges(
      seqnames = c(paste0('chr', Rle(chromosome))),
      ranges = c( IRanges(start = position, end = position  + nchar(ref) -1)),
      strand = c( "*"),  # SNPs are usually unstranded
      SNP_id = c(paste0('chr', snp)),
      REF =  DNAStringSet(ref),
      ALT =  DNAStringSet(alt)
    )
    
    # Specify the genome build (e.g., hg19 or hg38)
    genome(gr) <- "hg38"
    attr(gr, "genome.package") = "BSgenome.Hsapiens.UCSC.hg38"
    
    # Get the break result
    breakr_result = try(motifbreakR(snpList = gr, filterp = TRUE, threshold = sel_thres , # defines whether to filter by p-value instead of pct score
                       pwmList = subset(MotifDb, 
                                        dataSource %in% c("HOCOMOCOv11-core-A","hPDI", "jolma2013" , "stamlab","SwissRegulon", "JASPAR_CORE",'JASPAR_2014','jaspar2016', "jaspar2018" , 'cisbp_1.02', 'HOMER', 'HOCOMOCOv10' ))))  # previous run:
    # "JASPAR_CORE",'JASPAR_2014','jaspar2016', "jaspar2018" , 'cisbp_1.02', 'HOMER', 'HOCOMOCOv10'
    
    if(class(breakr_result) != 'try-error'){
    
    breakr_result_data = as.data.frame(breakr_result )
    breakr_result_data$motifPos = NULL
    breakr_result_data$altPos = NULL
    result_data = rbind(breakr_result_data, result_data) 
    print(i)
    print(snp)
    print(nrow(result_data))
    flush.console()
        }
    write.csv(result_data, paste0(data_path_meta, '/MOTIF_matches_meta_5ds_results_snps_breakr_280125', tested_snps, '_', sel_thres , '.csv'))
  } 


Warning message in prepareVariants(fsnplist = snpList, genome.bsgenome = genome.bsgenome, :
"The only variants included in the input set are indels, but legacy scoring was selected.
Legacy scoring is not availble for use with indels and will be disabled."
Warning message in motifbreakR(snpList = gr, filterp = TRUE, threshold = sel_thres, :
"No SNP/Motif Interactions reached threshold"


[1] 1
[1] "8:28092228:TTTG:T"
[1] 297504


Warning message in motifbreakR(snpList = gr, filterp = TRUE, threshold = sel_thres, :
"No SNP/Motif Interactions reached threshold"


[1] 2
[1] "6:31331551:G:C"
[1] 297504


Warning message in motifbreakR(snpList = gr, filterp = TRUE, threshold = sel_thres, :
"No SNP/Motif Interactions reached threshold"


...

[1] 114
[1] "1:226160095:C:CT"
[1] 298077


Warning message in prepareVariants(fsnplist = snpList, genome.bsgenome = genome.bsgenome, :
"The only variants included in the input set are indels, but legacy scoring was selected.
Legacy scoring is not availble for use with indels and will be disabled."
Warning message in motifbreakR(snpList = gr, filterp = TRUE, threshold = sel_thres, :
"No SNP/Motif Interactions reached threshold"


[1] 115
[1] "1:19868678:C:CT"
[1] 298077


Warning message in motifbreakR(snpList = gr, filterp = TRUE, threshold = sel_thres, :
"No SNP/Motif Interactions reached threshold"


[1] 116
[1] "1:69671928:A:G"
[1] 298077


In [ ]:
write.csv(result_data, paste0(data_path_meta, '/MOTIF_matches_meta_5ds_results_snps_breakr', '.csv'))

## Calculate the overlap with MOTIFs (searchseq)

In [ ]:
### Use searchseq to check for motifs around SNPs

In [ ]:
## Window Sizes

In [135]:
j  = 20

In [ ]:
### Alternative: initialize empty dataframe or load already calculated results to not recalculate

In [136]:
#motif_match = data.frame()

In [137]:
motif_match = read.csv(paste0(data_path_meta, '/MOTIF_matches_meta_5ds_results_snps_', tested_snps, '_', sel_thres, '_', j, '.csv'))

In [138]:
motif_match$X = NULL

In [139]:
head(motif_match,2)

,seqnames,source,feature,start,end,absScore,relScore,strand,ID,TF,class,siteSeqs,snp_id,sequence,window_size
,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
1,Wild-type,TFBS,TFBS,11,19,3.483203,0.8007783,-,MA0084.1,SRY,High-mobility group (HMG) domain factors,TTTAGCAAA,9:134944489:C:A,ATCCATCAAGTTTGCTAAACCACTTTCCCTCTGGAGCCATG,20
2,Wild-type,TFBS,TFBS,1,6,3.642925,0.8276184,+,MA0130.1,ZNF354C,C2H2 zinc finger factors,ATCCAT,9:134944489:C:A,ATCCATCAAGTTTGCTAAACCACTTTCCCTCTGGAGCCATG,20


In [140]:
nrow(snp_tests)

[1] 13454

In [141]:
length(unique(unique(motif_match$snp_id)))

[1] 13416

In [142]:
snp_tests = snp_tests[!snp_tests$snp_id %in% unique(motif_match$snp_id),]

In [143]:
nrow(snp_tests)

[1] 74

In [144]:
# Get human TF binding motifs from JASPAR
opts <- list(species = 9606)
motifs <- getMatrixSet(JASPAR2020, opts)

In [145]:
# Convert PFMatrixList to PWMatrixList
pwms <- toPWM(motifs)  # motifs is the PFMatrixList, we convert it to PWMatrixList

In [146]:
#str(pwms)

In [147]:
# Create a data frame with Matrix IDs and Gene Symbols (for mapping the tfs)
motif_info <- data.frame(
  matrix_id = sapply(pwms, ID),  # Get the JASPAR matrix ID
  gene_symbol = sapply(pwms, name)  # Get the corresponding gene name
)

In [148]:
##  general parameters for search
window_sizes <- c(20) # window around snp position to take into account for motif

In [149]:
### Iterate over all SNPs and check whether there is motif overlap at the position

In [150]:
#for(i in 1:nrow(co_egene_tfs)){
for(j in window_sizes){
    for(i in 1:nrow(snp_tests)){
        #data = co_egene_tfs[i,]
        data = snp_tests[i,]

        chr = paste0('chr', data$snp_chr)
        snp_position = as.numeric(data$snp_pos)
        snp = data$snp_id

        ## get the sequence of the snp
        snp_region <- GRanges(seqnames = chr, ranges = IRanges(start = snp_position - j, end = snp_position + j))
        sequence <- getSeq(BSgenome.Hsapiens.UCSC.hg38, snp_region)
        data$sequence = as.character(sequence) # save the sequence to the data

        ### Create wild-tipe and variant sequence
        wild_type_seq <- as.character(sequence)
        variant_seq <- wild_type_seq
        substr(variant_seq,j + 1,j + 1) <- data$allele ##TBD: check whether allele always the second information as assumed now in extraction

        data$variant_seq = variant_seq # save the variant sequence to the data

        ##  Convert to format to search for in JASPAR
        wild_type_dna <- DNAString(wild_type_seq)
        variant_dna <- DNAString(variant_seq)

        # Scan for TF binding sites in both sequences

        # Scan the wild-type sequence for TF binding sites
        wild_sites <- searchSeq(pwms, wild_type_dna, min.score = "80%", seqname = "Wild-type")
        ## TBD: add pvalues(wild_sites) --> to get p-values

        # Scan the variant sequence for TF binding sites
        variant_sites <- searchSeq(pwms, variant_dna, min.score = "80%", seqname = "Variant")

        # Get the TFS
        wild_sites_df <- as.data.frame(wild_sites)   # check whether directly extract the symbol
        variant_sites_df <- as.data.frame(variant_sites)

        # Replace matrix IDs in the result with gene symbols
        #wild_tfs_symbols <- motif_info$gene_symbol[match(wild_tfs, motif_info$matrix_id)]
        #variant_tfs_symbols <- motif_info$gene_symbol[match(variant_tfs, motif_info$matrix_id)]

        # Get Dataframe of the MOTIF results
        #wild_sites_df = writeGFF3(wild_sites)
        #variant_sites_df = writeGFF3(variant_sites)

        # Add the TF information
        #wild_sites_df$TF = wild_tfs_symbols
        #variant_sites_df$TF = variant_tfs_symbols

        # Add the information about snp
        wild_sites_df$snp_id = snp
        wild_sites_df$sequence = data$sequence
        variant_sites_df$snp_id = snp
        variant_sites_df$sequence = data$variant_seq 

        result_data = rbind(wild_sites_df, variant_sites_df)
        result_data$window_size = j

        #data$variant_motif_match = 0
        #data$variant_motif_match[data$coeGene %in% variant_tfs_symbols] = 1  # do the comparison now later

        #data$wild_motif_match = 0
        #data$wild_motif_match[data$coeGene %in% wild_tfs_symbols] = 1

        #data$motif_tfs = paste0(sort(unique(c(variant_tfs_symbols, wild_tfs_symbols))), collapse = ';')
        motif_match = rbind(motif_match, result_data)
        print(i)
        flush.console()
        }

    write.csv(motif_match, paste0(data_path_meta, '/MOTIF_matches_meta_5ds_results_snps_191124', tested_snps, '_', sel_thres, '_', j, '.csv'))
  }  
    

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74


In [ ]:
## Save the results

In [ ]:
write.csv(motif_match, paste0(data_path_meta, '/MOTIF_matches_meta_5ds_results.csv'))